In [ ]:
# data manipulation
import numpy as np
import pandas as pd
from pandas import DataFrame, read_excel, merge
import random as rnd

# plotting
import seaborn as sn
import matplotlib.pyplot as plt
#matplotlib inline

# machine learning
from numpy.core.umath_tests import inner1d

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from flask import Flask, request, render_template, send_from_directory
# Load libraries

from pandas.plotting import scatter_matrix
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.exceptions import NotFittedError
import json
import os

dataset = pd.read_excel (r'D:\Ashley\VendorData_Vis.xlsx')
print("Shape of dataset::{}".format(dataset.shape))
dataset.info()
dataset.head()

# Split-out validation dataset


X = dataset.iloc[:, [1,5,7,8]].values

Y = dataset.iloc[:,[0]].values

validation_size = 0.15
#seed = 10
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=100)

scoring = 'accuracy'

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_validation = sc.transform(X_validation)

# Spot Check Algorithms
models = []

models.append(('CART', DecisionTreeClassifier()))

results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=100)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    model.fit(X, Y)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
   

# Make predictions on validation dataset
dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 200)
dt.fit(X_train, Y_train)
dt.fit(X, Y)
predictions = dt.predict(X_validation)

print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

print(dt.predict([[52187,0,40,8]]))


import pickle
from flask_cors import CORS

app=Flask(__name__)
CORS(app)

@app.route('/vendor/<itemname>/<quantity>/<price>')
def vendor(itemname,quantity,price):
   
    Xnew = [[itemname, quantity, price, 1]]
    ynew = dt.predict(Xnew)
    print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))
    return "%s" % (ynew[0])


dataset = pd.read_excel (r'D:\Ashley\VendorData_Vis.xlsx')
print("Shape of dataset::{}".format(dataset.shape))
dataset.info()
dataset.head()

dataset['ItemNumber'] = dataset['ItemNumber'].apply(str)
dataset['ItemNumber'] = dataset['ItemNumber'].to_string()

data = pd.read_excel (r'D:\Ashley\VendorData_Vis.xlsx', dtype=str)
dataset['ItemNumber']=data['ItemNumber']
#dataset.info()

dataset.head()

#item = input('Enter your item number: ')
@app.route('/vendordata/<item>', methods =['GET'])
def generateimage(item):
#Both the following do the same thing
#sub_df = dataset.loc[dataset['ItemNumber'] == item]

    df=dataset[dataset['ItemNumber'].str.match(str(item))]
#Hence both are the same sub_df
#print(sub_df)
    print(df)
    if os.path.isfile(r"D:\Ashley\Visualisation\Quantity.png"):
        os.remove(r"D:\Ashley\Visualisation\Quantity.png")
    print('Item Number is '+ item)
    fig,(ax2) = plt.subplots(ncols=1)
    sn.barplot(data = df ,x='Vendor-No',y='Quantity Ordered',hue='Year',ax=ax2)
    fig.savefig(r"D:\Ashley\Visualisation\Quantity.png")# save the figure to file
    plt.close()

    if os.path.isfile(r"D:\Ashley\Visualisation\Price.png"):
        os.remove(r"D:\Ashley\Visualisation\Price.png")
    print('Item Number is '+ item)
    fig,(ax2) = plt.subplots(ncols=1)
    sn.barplot(data = df ,x='Vendor-No',y='Price',hue='Year',ax=ax2)
    fig.savefig(r"D:\Ashley\Visualisation\Price.png")   # save the figure to file
    plt.close()

    if os.path.isfile(r"D:\Ashley\Visualisation\DeliverySuccess.png"):
        os.remove(r"D:\Ashley\Visualisation\UI\DeliverySuccess.png")
    fig,(ax2) = plt.subplots(ncols=1)
    sn.barplot(data = df ,x='Vendor-No',y='Success Ratio',hue='Year',ax=ax2)
    fig.savefig(r"D:\Ashley\Visualisation\DeliverySuccess.png")  # save the figure to file
    plt.close()

    if os.path.isfile(r"D:\Ashley\Visualisation\DelayedDays.png"):
        os.remove(r"D:\Ashley\Visualisation\DelayedDays.png")
    fig,(ax2) = plt.subplots(ncols=1)
    sn.barplot(data = df ,x='Vendor-No',y='Delayed Days',hue='Year',ax=ax2)
    fig.savefig(r"D:\Ashley\Visualisation\DelayedDays.png")   # save the figure to file
    plt.close()

@app.route('/vendordata_raw/<username>', methods =['GET'])
def processjson(username):
    data = dataset.loc[dataset['ItemNumber'] == username]
    data.sort_values("Vendor-No", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')
    del data['ItemNumber']
    decimals = 2    
    data['Success Ratio'] = data['Success Ratio'].apply(lambda x: round(x, decimals))
    data["Month"] = data["Month"].map(str) + "/" + data["Year"].map(str)
    del data['Year']
    data=data.rename(columns={'Month': 'Date'})
    data_json = data.to_json(orient='records')

    return data_json
   
app.run()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  del sys.path[0]


Shape of dataset::(21129, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21129 entries, 0 to 21128
Data columns (total 9 columns):
Vendor-No                     21129 non-null int64
ItemNumber                    21129 non-null int64
Month                         21129 non-null int64
Year                          21129 non-null int64
Quantity Ordered              21129 non-null int64
Quantity Delivered On Time    21129 non-null int64
Success Ratio                 21129 non-null float64
Price                         21129 non-null float64
Delayed Days                  21129 non-null int64
dtypes: float64(2), int64(7)
memory usage: 1.5 MB
CART: 0.784731 (0.012006)
0.0003154574132492114
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
             precision    recall  f1-score   support

     600164       0.00      0.00      0.00        15
     600541       0.00      0.00      0.00        16
     600985       0.00

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Shape of dataset::(21129, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21129 entries, 0 to 21128
Data columns (total 9 columns):
Vendor-No                     21129 non-null int64
ItemNumber                    21129 non-null int64
Month                         21129 non-null int64
Year                          21129 non-null int64
Quantity Ordered              21129 non-null int64
Quantity Delivered On Time    21129 non-null int64
Success Ratio                 21129 non-null float64
Price                         21129 non-null float64
Delayed Days                  21129 non-null int64
dtypes: float64(2), int64(7)
memory usage: 1.5 MB
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2019 11:21:48] "GET /vendor/461/461/641 HTTP/1.1" 200 -


...

Item Number is 461


127.0.0.1 - - [19/Sep/2019 11:21:49] "GET /vendor/461/461/641 HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2019 11:21:49] "GET /vendordata_raw/461 HTTP/1.1" 200 -


X=['461', '461', '641', 1], Predicted=611538
      Vendor-No ItemNumber  Month  Year  Quantity Ordered  \
5447     611329        461      4  2018              1050   
5570     611329        461      2  2018               120   
5695     611538        461     11  2017               333   
5755     611538        461     12  2017               600   
5981     611538        461     10  2017               666   
6041     611329        461     10  2018               723   
6180     614448        461      5  2018               374   
6296     614448        461      7  2018               674   
6663     614448        461      8  2018               858   
6718     614448        461      4  2018                30   
6869     611329        461      7  2018               457   
7032     611329        461      5  2018               714   
7216     614448        461     11  2018               702   
7281     611538        461      1  2018               759   
8012     611329        461      9  2018 

[2019-09-19 11:21:50,043] ERROR in app: Exception on /vendordata/461 [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask_cors\extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 35, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app

Item Number is 461
